In [70]:
result_folder = "/home/yatin/hpchome/pratheek/PartialLabelLearning/results/lstm_flip"

In [71]:
import os,string
path = result_folder
path = os.path.normpath(path)
res = []
for root,dirs,files in os.walk(path, topdown=True):
    depth = root[len(path) + len(os.path.sep):].count(os.path.sep)
    if depth == 2:
        # We're currently two directories in, so all subdirs have depth 3
        res += [os.path.join(root, d) for d in dirs]
        dirs[:] = [] # Don't recurse any deeper
print(res)

['/home/yatin/hpchome/pratheek/PartialLabelLearning/results/lstm_flip/MSRCv2_flip/3layer/cc_loss_Adam_0.1_1e-06/0', '/home/yatin/hpchome/pratheek/PartialLabelLearning/results/lstm_flip/MSRCv2_flip/3layer/cc_loss_Adam_0.1_1e-06/1', '/home/yatin/hpchome/pratheek/PartialLabelLearning/results/lstm_flip/MSRCv2_flip/3layer/cc_loss_Adam_0.1_1e-06/2', '/home/yatin/hpchome/pratheek/PartialLabelLearning/results/lstm_flip/MSRCv2_flip/3layer/cc_loss_Adam_0.1_1e-06/3', '/home/yatin/hpchome/pratheek/PartialLabelLearning/results/lstm_flip/MSRCv2_flip/3layer/cc_loss_Adam_0.1_1e-06/4', '/home/yatin/hpchome/pratheek/PartialLabelLearning/results/lstm_flip/MSRCv2_flip/3layer/cc_loss_Adam_0.1_1e-06/6', '/home/yatin/hpchome/pratheek/PartialLabelLearning/results/lstm_flip/MSRCv2_flip/3layer/cc_loss_Adam_0.1_1e-06/9', '/home/yatin/hpchome/pratheek/PartialLabelLearning/results/lstm_flip/MSRCv2_flip/3layer/cc_loss_Adam_0.1_1e-06/7', '/home/yatin/hpchome/pratheek/PartialLabelLearning/results/lstm_flip/MSRCv2_fli

In [72]:
columns = ['dataset','technique','fold','real_test_acc','train_IOU','train_IOU_neg','val_IOU','val_IOU_neg','test_IOU','test_IOU_neg','surrogate_test_acc', 'surrogate_train_acc', 
'real_train_acc', 'surrogate_val_acc', 'real_val_acc',  'train_confidence', 'val_confidence', 'test_confidence','best_epoch']

In [73]:
import pandas as pd
import time 
data = pd.DataFrame(columns=columns) 
count = 0
for folder in res:
    
    logfile = os.path.join(folder, "logs","log.json")
    if(not(os.path.exists(logfile))):
        continue
        
    #if("cc_loss" in logfile):
    if("0.1" in logfile):
        continue
    #if("cc_loss" in logfile):
    #    if("1_1e-05" in logfile):
    #        continue
    #if(not("/2/" in logfile)):
    #    continue
    #if(not("cc_loss") in logfile):
    #    if(not("pretrain") in logfile):
    #        continue
    
    count+=1
    #print("created: %s" % time.ctime(os.path.getctime(logfile)))
    df = pd.read_json(logfile, lines=True)
    df = df[df["epoch"] == -1]
    #print(df['real_test_acc'])
    #print(df.columns.values)
    names = folder.split("/")
    for index, row in df.iterrows():
        
        if(row["epoch"] == -1):
            data.loc[count] = row
            data.loc[count,"dataset"] = names[-4]
            #data.loc[count,"model"] = names[-3]
            data.loc[count,"technique"] = names[-2]
            data.loc[count,"fold"] = names[-1]
            
            #print(data)
print(count)
#data.to_csv('fold_2.csv')

20


In [6]:
techniques = data[["technique","fold","dataset"]].values.tolist()

dic = {}
techs = 0
for i in techniques:
    if(("lstm" in i[0]) and ("detach" in i[0]) and ("MSRCv2_lstm") in i[2]):
        techs+=1
        tokens = i[0].split("_")
        
        wd = tokens[-3]
        lr = tokens[-4]
        fold = i[1]
        opt = tokens[-5]
        pretrain = tokens[-2]
        dic[(opt,lr,wd,fold,pretrain)] = 1
count = 0
missing = 0
for lr in ['0.01','0.1']:
    for wd in [0.001, 0.0001, 1e-05, 1e-06]:
        for opt in ['Adam','SGD']:
            for fold in range(10):
                for pretrain in [0,100]:
                    count+=1
                    if(not((opt,str(lr),str(wd),str(fold),str(pretrain)) in dic)):
                        #print("weighted_loss_xy_lstm_{}_{}_{}_{}_NONORM/{}".format(opt,lr,wd,pretrain,fold))
                        missing +=1
print(count)
print(missing)
print(techs)

320
320
0


In [343]:
data[columns].groupby(['dataset','technique','fold']).mean()

real_test_acc  \
dataset                technique                              fold                  
BirdSong               cc_loss_Adam_0.0001_0.001              0         73.146293   
                       cc_loss_Adam_0.0001_1e-05              0         72.945892   
                       cc_loss_Adam_0.001_0.001               0         73.346693   
                       cc_loss_Adam_0.001_1e-06               0         71.743487   
                       cc_loss_Adam_0.01_0.0001               0         74.348697   
...                                                                           ...   
lost_noisy_annotator_p weighted_loss_xy_lstm_SGD_0.0001_1e-05 0         51.785714   
                       weighted_loss_xy_lstm_SGD_0.001_1e-06  0         66.071429   
                       weighted_loss_xy_lstm_SGD_0.1_0.0001   0         68.750000   
                       weighted_loss_xy_lstm_SGD_0.1_1e-05    0         66.071429   
                       weighted_loss_xy_lstm_SGD_0.1_1e-06    0         68.750000   

                                                                    train_IOU  \
dataset                technique                              fold              
BirdSong               cc_loss_Adam_0.0001_0.001              0           NaN   
                       cc_loss_Adam_0.0001_1e-05              0           NaN   
                       cc_loss_Adam_0.001_0.001               0           NaN   
                       cc_loss_Adam_0.001_1e-06               0           NaN   
                       cc_loss_Adam_0.01_0.0001               0           NaN   
...                                                                       ...   
lost_noisy_annotator_p weighted_loss_xy_lstm_SGD_0.0001_1e-05 0           NaN   
                       weighted_loss_xy_lstm_SGD_0.001_1e-06  0           NaN   
                       weighted_loss_xy_lstm_SGD_0.1_0.0001   0           NaN   
                       weighted_loss_xy_lstm_SGD_0.1_1e-05    0           NaN   
                       weighted_loss_xy_lstm_SGD_0.1_1e-06    0           NaN   

                                                                    val_IOU  \
dataset                technique                              fold            
BirdSong               cc_loss_Adam_0.0001_0.001              0         NaN   
                       cc_loss_Adam_0.0001_1e-05              0         NaN   
                       cc_loss_Adam_0.001_0.001               0         NaN   
                       cc_loss_Adam_0.001_1e-06               0         NaN   
                       cc_loss_Adam_0.01_0.0001               0         NaN   
...                                                                     ...   
lost_noisy_annotator_p weighted_loss_xy_lstm_SGD_0.0001_1e-05 0         NaN   
                       weighted_loss_xy_lstm_SGD_0.001_1e-06  0         NaN   
                       weighted_loss_xy_lstm_SGD_0.1_0.0001   0         NaN   
                       weighted_loss_xy_lstm_SGD_0.1_1e-05    0         NaN   
                       weighted_loss_xy_lstm_SGD_0.1_1e-06    0         NaN   

                                                                    test_IOU  \
dataset                technique                              fold             
BirdSong               cc_loss_Adam_0.0001_0.001              0          NaN   
                       cc_loss_Adam_0.0001_1e-05              0          NaN   
                       cc_loss_Adam_0.001_0.001               0          NaN   
                       cc_loss_Adam_0.001_1e-06               0          NaN   
                       cc_loss_Adam_0.01_0.0001               0          NaN   
...                                                                      ...   
lost_noisy_annotator_p weighted_loss_xy_lstm_SGD_0.0001_1e-05 0          NaN   
                       weighted_loss_xy_lstm_SGD_0.001_1e-06  0          NaN   
                       weighted_loss_xy_lstm_SGD_0.1_0.0001   0          NaN   


In [227]:
def custom_sort(tup):
    #print(tup)
    print(tup)
    if('lost' in tup[1]):
        a = 40
    elif('MSRCv2' in tup[1]):
        a = 10
    elif('BirdSong' in tup[1]):
        a = 20
    else:
        a = 30
        
    if('one' in tup[1]):
        a += 1
    elif('two' in tup[1]):
        a += 2
    elif('flip' in tup[1]):
        a += 3
    elif('noisy' in tup[1]):
        a += 4
    else:
        a += 5
    return ( a,tup[0])


,dataset,technique,fold,real_test_acc,train_IOU,train_IOU_neg,val_IOU,val_IOU_neg,test_IOU,test_IOU_neg,surrogate_test_acc,surrogate_train_acc,real_train_acc,surrogate_val_acc,real_val_acc,train_confidence,val_confidence,test_confidence,best_epoch
1,lost_noisy_annotator_p,cc_loss_Adam_0.001_0.0001,0,71.428571,NaN,NaN,NaN,NaN,NaN,NaN,82.142857,100.000000,87.946429,86.607143,80.357143,0.996920,0.917440,0.875374,1271
10,lost_noisy_annotator_p,cc_loss_Adam_0.001_1e-05,0,67.857143,NaN,NaN,NaN,NaN,NaN,NaN,77.678571,100.000000,87.834821,86.607143,82.142857,0.997555,0.928105,0.908326,1391
16,lost_noisy_annotator_p,cc_loss_SGD_0.0001_1e-05,0,40.178571,NaN,NaN,NaN,NaN,NaN,NaN,52.678571,75.446429,55.245536,64.285714,52.678571,0.380222,0.357375,0.355152,1317
17,lost_noisy_annotator_p,weighted_loss_xy_SGD_0.0001_1e-05,0,52.678571,NaN,NaN,NaN,NaN,NaN,NaN,65.178571,89.397321,70.647321,80.357143,71.428571,0.588546,0.533412,0.500174,842
18,lost_noisy_annotator_p,weighted_loss_xy_SGD_0.0001_0.001,0,54.464286,NaN,NaN,NaN,NaN,NaN,NaN,66.964286,89.620536,70.982143,81.250000,71.428571,0.599977,0.543434,0.507670,1268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2627,lost,cc_loss_SGD_0.1_0.001,0,69.642857,NaN,NaN,NaN,NaN,NaN,NaN,78.571429,100.000000,91.183036,84.821429,77.678571,0.972744,0.866620,0.841397,1486
2628,lost,weighted_loss_xy_SGD_0.0001_0.0001,0,52.678571,NaN,NaN,NaN,NaN,NaN,NaN,65.178571,89.397321,70.647321,80.357143,71.428571,0.589496,0.534223,0.500778,877
2630,lost,weighted_loss_xy_lstm_SGD_0.0001_0.001,0,51.785714,NaN,NaN,NaN,NaN,NaN,NaN,63.392857,89.285714,68.638393,80.357143,68.750000,0.572772,0.517730,0.486336,1394
2636,lost,weighted_loss_xy_Adam_0.01_0.001,0,30.357143,NaN,NaN,NaN,NaN,NaN,NaN,60.714286,95.312500,51.674107,68.750000,34.821429,0.906081,0.774859,0.788931,756


In [28]:
#selected_cols = ['fully_supervised',
#'cc_loss',
#'iexplr_loss',
#'weighted_loss_xy',
#'weighted_loss_xy_iexplr',
##'weighted_loss_y',
#'weighted_loss_y_iexplr',
#                'weighted_fully_supervised',
#'weighted_fully_supervised_iexplr',  ]

a = data[columns].groupby(['dataset','technique','fold']).mean().reset_index()
a = a.loc[a['technique'].str.contains('detach') | a['technique'].str.contains('cc_loss')]
#a = a.loc[a['fold'].str.contains('0')]

#a = a.loc[a['technique'].str.contains('fully')]
#print(a.technique)
#a = (a[a['technique'].isin(selected_cols)])
b = a.set_index(['dataset','technique','fold']).unstack(level=0)
#b = b[['real_test_acc','surrogate_val_acc','real_val_acc']]
b = b.reindex(sorted(b.columns), axis=1)

b.to_csv('results_xy.csv')

DataError: No numeric types to aggregate

In [74]:

c = data[columns]

c = c.loc[c['technique'].str.contains('pretrain') | c['technique'].str.contains('cc_loss') | c['technique'].str.contains('freeze') | c['technique'].str.contains('pq') ]
c = c.loc[c['dataset'].str.endswith('MSRCv2_flip')]
c = c.loc[~c['technique'].str.contains('_1_')]

cc = c.loc[c['technique'].str.contains('cc_loss')]
cc = cc.loc[~cc['technique'].str.contains('1e-05')]
cc = cc.loc[~cc['technique'].str.contains('1_0.001_')]

cc['tech'] = 'cc'
print(cc['technique'].unique())
print(cc.shape)
cc = cc.loc[cc.groupby(['dataset','tech','fold'])['surrogate_val_acc'].idxmax()]


lstm = c.loc[c['technique'].str.contains('weighted_loss_xy_lstm_iexplr')]
#print(lstm)
lstm = lstm.loc[~lstm['technique'].str.contains('fully')]

lstm_pq = lstm.loc[lstm['technique'].str.endswith('_100_pq')]
lstm_freeze = lstm.loc[lstm['technique'].str.contains('_100_freeze')]
lstm_pretrain = lstm.loc[lstm['technique'].str.contains('_100_pretrain')]
print(lstm_freeze.shape)
print(lstm_pretrain.shape)
lstm_pq['tech'] = 'lstm'
lstm_freeze['tech'] = 'lstm_freeze'
lstm_pretrain['tech'] = 'lstm_freeze+pretrain'

lstm_pq = lstm_pq.loc[lstm_pq.groupby(['dataset','tech','fold'])['surrogate_val_acc'].idxmax()]
lstm_freeze = lstm_freeze.loc[lstm_freeze.groupby(['dataset','tech','fold'])['surrogate_val_acc'].idxmax()]
lstm_pretrain = lstm_pretrain.loc[lstm_pretrain.groupby(['dataset','tech','fold'])['surrogate_val_acc'].idxmax()]

c = pd.concat([cc, lstm_pq, lstm_freeze, lstm_pretrain])

#c = c[['real_test_acc','surrogate_val_acc','real_val_acc']]
c = c.reindex(sorted(c.columns), axis=1)
#c = c[c['dataset'] == 'MSRCv2_lstm']
c = c.pivot(index="tech", columns=["dataset","fold"])
print(c.head())
#c = c.set_index(['dataset','tech','fold']).unstack(level=0)

c.to_csv('results_xy_lstm_val_surr.csv')

['cc_loss_Adam_0.01_1e-06']
(10, 20)
(0, 19)
(10, 19)
                      best_epoch                                         ...  \
dataset              MSRCv2_flip                                         ...   
fold                           0    1    2  3    4   5    6   7  8    9  ...   
tech                                                                     ...   
cc                           762  709  771  9  792   6   16  20  8  798  ...   
lstm_freeze+pretrain         209  768    1  1    1  54  972   1  1   67  ...   

                     val_confidence                                          \
dataset                 MSRCv2_flip                                           
fold                              0         1         2         3         4   
tech                                                                          
cc                         0.899774  0.915350  0.927271  0.722689  0.908199   
lstm_freeze+pretrain       0.907027  0.911633  0.927271  0.722689  0.9

In [20]:
import pandas as pd
data = pd.DataFrame(columns=columns) 
count = 0
for folder in res:
    
    logfile = os.path.join(folder, "logs","log.json_lstm")
    if(not(os.path.exists(logfile))):
        continue
    df = pd.read_json(logfile, lines=True)
    df = df[df["epoch"] == -2]
    #print(df.columns.values)
    names = folder.split("/")
    for index, row in df.iterrows():
        
        if(row["epoch"] == -2):
            data.loc[count] = row
            data.loc[count,"dataset"] = names[-4]
            #data.loc[count,"model"] = names[-3]
            data.loc[count,"technique"] = names[-2]
            data.loc[count,"fold"] = names[-1]
            count+=1


c = data[columns]
print(data)
c = c.loc[c['technique'].str.contains('pretrain') | c['technique'].str.contains('cc_loss') | c['technique'].str.contains('freeze') | c['technique'].str.contains('pq') ]
c = c.loc[c['dataset'].str.endswith('MSRCv2')]
c = c.loc[~c['technique'].str.contains('_1_')]

cc = c.loc[c['technique'].str.contains('cc_loss')]
cc = cc.loc[~cc['technique'].str.contains('1e-05')]
cc = cc.loc[~cc['technique'].str.contains('1_0.001_')]

cc['tech'] = 'cc'
cc = cc.loc[cc.groupby(['dataset','tech','fold'])['real_val_acc'].idxmax()]


lstm = c.loc[c['technique'].str.contains('weighted_loss_xy_lstm_iexplr')]
#print(lstm)
lstm = lstm.loc[~lstm['technique'].str.contains('fully')]

lstm_pq = lstm.loc[lstm['technique'].str.endswith('_100_pq')]
lstm_freeze = lstm.loc[lstm['technique'].str.contains('_100_freeze')]
lstm_pretrain = lstm.loc[lstm['technique'].str.contains('_100_pretrain')]

lstm_pq['tech'] = 'lstm'
lstm_freeze['tech'] = 'lstm_freeze'
lstm_pretrain['tech'] = 'lstm_freeze+pretrain'

lstm_pq = lstm_pq.loc[lstm_pq.groupby(['dataset','tech','fold'])['real_val_acc'].idxmax()]
lstm_freeze = lstm_freeze.loc[lstm_freeze.groupby(['dataset','tech','fold'])['real_val_acc'].idxmax()]
lstm_pretrain = lstm_pretrain.loc[lstm_pretrain.groupby(['dataset','tech','fold'])['real_val_acc'].idxmax()]

c = pd.concat([cc, lstm_pq, lstm_freeze, lstm_pretrain])

c = c.reindex(sorted(c.columns), axis=1)
#c = c[c['dataset'] == 'MSRCv2_lstm']
c = c.pivot(index="tech", columns=["dataset","fold"])
print(c.head())
#c = c.set_index(['dataset','tech','fold']).unstack(level=0)

c.to_csv('results_xy_lstm_val_comb.csv')

   dataset                                          technique fold  \
0   MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.1_0.0001_1...    1   
1   MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.1_0.0001_1...    3   
2   MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.1_0.0001_1...    9   
3   MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.001_0.0001...    4   
4   MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.1_0.0001_1...    1   
..     ...                                                ...  ...   
80  MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.01_0.0001_...    0   
81  MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.01_0.0001_...    2   
82  MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.01_0.0001_...    4   
83  MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.01_0.0001_...    6   
84  MSRCv2  weighted_loss_xy_lstm_iexplr_Adam_0.01_0.0001_...    8   

    real_test_acc  train_IOU  train_IOU_neg   val_IOU  val_IOU_neg  test_IOU  \
0       42.857143  -1.000000      -1.000000 -1.000000    -1.000000 -1.000000   

/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/yatin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p